In [2]:
from typing import Tuple, List, cast

import pandas as pd
import numpy as np

from consts import get_region_from_city
from utils import batched

pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [8]:
# comparing q3, q4


def prepare_sales_sheet(sheet_path: str):
    sales = pd.read_excel(sheet_path)
    sales = sales.assign(UnitProfit=lambda x: x["Gross Margin"] / x["Unit Sold"])
    return sales


def compare_sales(
    comparison_df: pd.DataFrame, sales_params: List[Tuple[pd.DataFrame, str]]
):
    for b in cast(
        List[Tuple[Tuple[pd.DataFrame, str], Tuple[pd.DataFrame, str]]],
        batched(sales_params, 2),
    ):
        s1, s2 = b
        if s2 is not None:
            s1_df, s1_name = s1
            s2_df, s2_name = s2
            # fill na with 0
            s1_df.fillna(0, inplace=True)
            s2_df.fillna(0, inplace=True)
            merged_sales = s2_df.set_index("Brand").join(
                s1_df.set_index("Brand"), lsuffix="_s2", rsuffix="_s1"
            )
            merged_sales.fillna(0, inplace=True)
            comparison_df[f"{s2_name}-{s1_name}_UnitProfit"] = (
                merged_sales["UnitProfit_s2"] - merged_sales["UnitProfit_s1"]
            )
            comparison_df[f"{s2_name}-{s1_name}_UnitSold"] = (
                merged_sales["Unit Sold_s2"] - merged_sales["Unit Sold_s1"]
            )

            comparison_df[f"{s2_name}-{s1_name}_GrossMargin"] = (
                merged_sales["Gross Margin_s2"] - merged_sales["Gross Margin_s1"]
            )

            # set gross margin < 0 as 0, if UnitSold is negative, that means the product was discontinued
            comparison_df[f"{s2_name}-{s1_name}_GrossMargin"] = np.where(
                comparison_df[f"{s2_name}-{s1_name}_UnitSold"] < 0,
                0,
                comparison_df[f"{s2_name}-{s1_name}_GrossMargin"],
            )
        else:
            pass
    return comparison_df


q3_sales = prepare_sales_sheet("sheets/qtr_4/Sales-Q3.xlsx")
q4_sales = prepare_sales_sheet("sheets/qtr_5/Sales-Q4.xlsx")

sales_to_compare: List[Tuple[pd.DataFrame, str]] = [(q3_sales, "q3"), (q4_sales, "q4")]
sales_comparison_df = pd.DataFrame()
compare_sales(sales_comparison_df, sales_to_compare)
sales_comparison_df.to_excel("output_sheets/sales_comparison.xlsx")

,q4-q3_UnitProfit,q4-q3_UnitSold,q4-q3_GrossMargin
Brand,,,
2. Velo Pro,-6.81,231.00,119285.93
2. Velo Nature,-682.66,-31.00,0.00
1. Velo Terrain,-691.94,-30.00,0.00
Velo Nature Max,598.22,134.00,80161.22
Velo Terrain XL,556.21,56.00,31147.53
Total,-39.62,360.00,188674.18
